# Homework
In the workshop, we extracted contents from two pages in notion titled "Workshop: Benefits and Perks" and "Workshop: Working hours, PTO, and Vacation".

Repeat the same process for a third page titled "Homework: Employee handbook" (hidden from public view, but accessible via API key):

In [1]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

1. Modify the REST API source to extract only this page.
2. Write the output into a separate table called "homework".
3. Remember to update the table name in all cells where you connect to a lancedb table.

In [28]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="homework")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "Homework: Employee handbook",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="homework")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': False}, {'name': 'schema', 'data_type': 'text', 'nullable': False}]
_dlt_loads
[{'name': 'load_id', 'da

In [37]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

data = dbtable.to_pandas()
data

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.024265619, 0.04746082, -0.011796436, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-16 19:07:31.751376+00:00,1721156850.2986174,qgyJszmFES7bMA
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.04966163, 0.10853516, -0.009762589, -0.036...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-16 19:07:31.754750+00:00,1721156850.2986174,XKFwhHpr2+kjrg
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316319, 0.17331506, 0.025351718, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-16 19:07:31.754928+00:00,1721156850.2986174,LzOjeOl4n5rhNg
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.10974315, 0.10586077, 0.0032906013, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-16 19:07:31.755092+00:00,1721156850.2986174,6/Oa1JW0ECxbyg
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.052423332, -0.06457594, 0.065862976, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-16 19:07:31.755260+00:00,1721156850.2986174,pl5euX9m/GFMkQ
5,434b71e9-a11a-519d-a9fe-e3ade78d47d6,"[0.0005233448, -0.054883398, 0.043573365, -0.0...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-16 19:07:31.755435+00:00,1721156850.2986174,b4LMMqPm2GlO+g
6,17816363-54b7-5ba7-b8d5-06d871a25414,"[0.03802629, -0.021509668, 0.047527827, 0.0647...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-16 19:07:31.755609+00:00,1721156850.2986174,b67olc8GRgL3MQ
7,2a434cf9-09d9-5514-a88b-02977f2f953e,"[-0.058588073, -0.07540443, 0.033775173, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-16 19:07:31.755783+00:00,1721156850.2986174,lOcqsNwDQItf2w
8,5f9384fa-7f98-5f52-a06e-05b05f42f69a,"[-0.01359925, 0.004753045, 0.024835167, 0.0159...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-16 19:07:31.755963+00:00,1721156850.2986174,zTgbYugRnRuxXg
9,42af72f6-9db7-54a2-87b2-d466169078ff,"[0.03206087, 0.02424462, 0.00847135, 0.0317907...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-16 19:07:31.756121+00:00,1721156850.2986174,i+VccYrg25tplw


Now, answer the following questions:

## Q1. Rows in LanceDB
How many rows does the lancedb table "notion_pages__homework" have?

14
15
16
**17**

## Q2. Running the Pipeline: Last edited time
In the demo, we created an incremental dlt resource rest_api_notion_incremental that keeps track of last_edited_time. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column last_edited_time of the table)

In [40]:
data['last_edited_time'].max()

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

+ Timestamp('2024-07-05 22:34:00+0000', tz='UTC') (OR "2024-07-05T22:34:00.000Z")
+ **Timestamp('2024-07-05 23:33:00+0000', tz='UTC') (OR "2024-07-05T23:33:00.000Z")**
+ Timestamp('2024-07-05 23:52:00+0000', tz='UTC') (OR "2024-07-05T23:52:00.000Z")
+ Timestamp('2024-07-05 22:56:00+0000', tz='UTC') (OR "2024-07-05T22:56:00.000Z")

## Q3. Ask the Assistant
Find out with the help of the AI assistant: how many PTO days are the employees entitled to in a year?

20
25
**30**
35

In [33]:
import ollama

In [34]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [35]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [41]:
main()

You:  how many PTO days are the employees entitled to in a year?


Assistant: Hi! Based on the provided context, it seems like employees are entitled to 30 days of paid time off (PTO) per year. However, if they have not accrued enough PTO or exceeded their allotted amount for the year, they may be able to carry over up to 25 additional days from one year to the next. It is recommended that employees consult with HR or their manager before taking any time off so that appropriate approvals can be obtained and schedules can be coordinated accordingly. Additionally, it seems like there are certain holidays where employees may receive premium pay for working on those days, but if they are non-exempt employees, they should consult with HR or their manager to determine if this is applicable in their case. Finally, exempt employees who work on a holiday will be granted an additional day of PTO that must be taken within 12 months after the holiday.


KeyboardInterrupt: Interrupted by user